In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import urllib
import requests
import regex as re
from bs4 import BeautifulSoup

import gutenberg_cleaner as gclean

import nltk
from nltk.tokenize import word_tokenize

from gensim.models import Word2Vec

<IPython.core.display.Javascript object>

In [3]:
# Parameters
book_list = "../data/book_list.csv"

REMOVE_WORDS = ["[deleted]"]
chapter_regex = re.compile("(I.)[\s\w\\n]*(II).[\s\w\\n]*(III).[\s\w\\n]*")

<IPython.core.display.Javascript object>

In [5]:
# Get list of books and their links
df = pd.read_csv(book_list)

<IPython.core.display.Javascript object>

# Get Text from Web

In [6]:
def clean_paragraphs(par):

    # Replace special characters
    par = par.replace("\n", " ")
    par = par.replace("\\", "")

    # Remove leading whitespace
    par = par.lstrip()

    return par

<IPython.core.display.Javascript object>

In [13]:
X = []
y = []
stop_words = set(stopwords.words("english"))

<IPython.core.display.Javascript object>

In [14]:
for _, row in df.iterrows():
    label = row.get("author")
    target_url = row.get("link")

    # Get text
    soup_html = urllib.request.urlopen(target_url).read()
    text = str(BeautifulSoup(soup_html))
    text_clean = gclean.super_cleaner(text)

    # Separate into paragraphs
    paragraphs = text_clean.split("\n\n")
    paragraphs = [item for item in paragraphs if item not in REMOVE_WORDS]
    paragraphs = [par for par in paragraphs if re.search(chapter_regex, par) is None]

    # Clean paragraphs
    paragraphs = list(map(lambda x: clean_paragraphs(x), paragraphs))

    # Append to data
    X.extend(paragraphs)
    y.extend([label] * len(paragraphs))

<IPython.core.display.Javascript object>

In [18]:
# Create an output set
# out_data = pd.DataFrame({"text": X, "label": y})
# out_data.to_csv("../data/clean_data.csv", index=False)

<IPython.core.display.Javascript object>

# Quick Descriptive Stats

In [19]:
out_data.groupby("label").count()

,text
label,
"Austen, Jane",7607
"Christie, Agatha",12354
"Dickens, Charles",25037
"Dostoyevsky, Fyodor",15421
"Hugo, Victor",23405
"London, Jack",8340
"Stowe, Harriett Beecher",12794
"Twain, Mark",10837


<IPython.core.display.Javascript object>

In [20]:
out_data.loc[:, "text_tokenized"] = [word_tokenize(text) for text in out_data["text"]]

<IPython.core.display.Javascript object>

In [21]:
out_data.loc[:, "text_tokenized_count"] = [
    len(tokenized) for tokenized in out_data["text_tokenized"]
]

<IPython.core.display.Javascript object>

In [22]:
out_data.groupby("label")["text_tokenized_count"].mean()

label
Austen, Jane               84.494282
Christie, Agatha           33.156144
Dickens, Charles           58.157806
Dostoyevsky, Fyodor        65.097140
Hugo, Victor               49.331895
London, Jack               63.851679
Stowe, Harriett Beecher    67.888542
Twain, Mark                72.619544
Name: text_tokenized_count, dtype: float64

<IPython.core.display.Javascript object>